# 📘 Notebook: Modular Multi-Agent Architecture with LLMs

This notebook demonstrates how to structure and execute modular components using agents, orchestrators, and LLMs. 
It includes an example of how a user can interact with specialized agents through a defined module pipeline.


## 🔧 Setup Agent Framework Components

We import the core components needed to build and coordinate agents, including `User`, `Agent`, `Module`, and `Orchestrator`.


In [1]:
from agents import User, Agent, Module, Orchestrator
from agents.scripted_users import UserModule
import os

c:\Users\FS-Ma\OneDrive\Documents\projects\arara\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## ⚙️ LLM Configuration and Module Setup

This block defines the configuration for the LLM and sets up modules that the agent system will use. 
Modules can be chained, parallelized, or triggered based on the orchestration logic.


In [2]:
# LLM configuration
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

# User agent
user = UserModule(
    description="You are a human user looking for content suggestions such as books or movies."
)

# Agent that talks directly to the user
vendedor = Agent(
    name="vendedor",
    llm_config=llm_config,
    system_message="""
    You are a helpful assistant engaging in friendly conversation.
    If the user expresses interest in a recommendation, trigger the 'content_orchestrator' module.
    Do not make any recommendations yourself.
""",
    description="Converses with the user and activates the recommendation module when needed."
)

# Tool: content recommender function
def get_content_recommendation(preference: str) -> str:
    """
    Returns a single content recommendation based on the user's preference.

    Parameters:
        preference (str): Either 'book' or 'movie'

    Returns:
        str: A recommended title
    """
    preference = preference.strip().lower()

    if preference == "book":
        return "Sapiens: A Brief History of Humankind – Yuval Noah Harari"
    elif preference == "movie":
        return "Inception – directed by Christopher Nolan"
    else:
        return "Please specify either 'book' or 'movie'."

# Agent: makes the recommendation
recommender_agent = Agent(
    name="recommender_agent",
    llm_config=llm_config,
    system_message="""
    You are a content recommendation agent.
    Your task is to recommend a book or movie based on the user request.
    Always use the 'get_content_recommendation' tool.
    Do not make up recommendations. Return only what the tool provides.
""",
    description="Recommends a book or movie using a fixed tool.",
    tools=[get_content_recommendation],
    reflect_on_tool_use=True,
)

# Agent: explains the recommendation
explainer_agent = Agent(
    name="explainer_agent",
    llm_config=llm_config,
    system_message="""
    You are an explanation agent.
    Your job is to explain *why* the recommended content (book or movie) matches the user's preference.
    Use the user's preference and the content item to generate a clear, short, and informative explanation.
    Do not recommend or invent new items.
""",
    description="Explains why the recommendation fits the preference."
)



allowed_speaker_transitions_dict = {
    recommender_agent: [explainer_agent],
}
# Module: internal logic of the recommendation
rec_module = Module(
    admin_name="content_module",
    agents=[recommender_agent, explainer_agent],
    speaker_selection_method="round_robin",
    allowed_or_disallowed_speaker_transitions=allowed_speaker_transitions_dict,
    speaker_transitions_type="allowed",
)

# Entry point to start the conversation flow
rec_orc = Orchestrator(
    name="rec_orchestrator",
    module=rec_module,
    llm_config=llm_config,
    system_message="Só repasse a mensagem.",
    description="Orchestrates the content recommendation process.",
)

main_module = Module(
    admin_name="main_module",
    agents=[user, rec_orc, vendedor],
    speaker_selection_method="auto",
    allow_repeat_speaker=False,
    #allowed_or_disallowed_speaker_transitions=speaker_transitions_dict,
    #speaker_transitions_type="allowed",
)

# Orchestrator to manage the conversation flow
arara_orc = Orchestrator(
    name = "main_orchestrator",
    module=main_module,
    llm_config=llm_config,
    system_message="""
    You are the main orchestrator of the conversation.
    Start by engaging the user and then let the 'vendedor' agent take over.
    The 'vendedor' will trigger the content recommendation module when appropriate.
""",
)

## 💬 Example Interaction Between User and Agent

This cell initiates a conversation between the user and a selected agent module. 
It shows how user inputs are processed and passed through the agent architecture.


In [ ]:
chat_result =   user.talk_to(
    arara_orc, message="Gostaria de um filme",
    cache=None
)

user_test ⟶ main_orchestrator:
Gostaria de um filme

Next speaker: vendedor

vendedor ⟶ main_orchestrator:
Um filme, é uma ótima escolha! Estou ansioso para ajudá-lo a encontrar algo que você vai gostar. Posso ajudá-lo a encontrar uma boa opção. Você gostaria de uma recomendação?

Next speaker: user_test

user_test ⟶ main_orchestrator:
Sim, de um filme

Next speaker: rec_orchestrator


Next speaker: recommender_agent

recommender_agent ⟶ rec_orchestrator:
🛠️ Suggested Function Call: get_content_recommendation
 Arguments:
 {
   "preference": "movie"
 }
recommender_agent ⟶ rec_orchestrator:
✅ Function Execution Result: get_content_recommendation
 Result:
 Inception – directed by Christopher Nolan
recommender_agent ⟶ rec_orchestrator:
Here is your movie recommendation: Inception – directed by Christopher Nolan

Next speaker: explainer_agent

explainer_agent ⟶ rec_orchestrator:
You'll love Inception because you appreciate complex, thought-provoking plots. Christopher Nolan's direction weav

In [4]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': 'Gostaria de um filme', 'role': 'assistant', 'name': 'user_test'}, {'content': 'Um filme, é uma ótima escolha! Estou ansioso para ajudá-lo a encontrar algo que você vai gostar. Posso ajudá-lo a encontrar uma boa opção. Você gostaria de uma recomendação?', 'name': 'vendedor', 'role': 'user'}, {'content': 'Sim, de um filme', 'role': 'assistant', 'name': 'user_test'}, {'content': "You'll love Inception because you appreciate complex, thought-provoking plots. Christopher Nolan's direction weaves together multiple layers of reality, keeping you engaged and curious about what's real and what's a dream. The film's intricate storyline and clever plot twists will appeal to your taste for intellectually stimulating cinema.", 'name': 'rec_orchestrator', 'role': 'user'}, {'content': 'Agora gostaria de um livro', 'role': 'assistant', 'name': 'user_test'}, {'content': 'Um livro, é uma ótima escolha! Estou ansioso para ajudá-lo a encontrar algo que v